In [1]:
import numpy as np
import pandas as pd

In [18]:
def preprocess(df, start_year, end_year, var):
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
    df.drop(['height'], inplace=True, axis=1)
    df['time'] = pd.to_datetime(df['time'])
    df['month'] = df['time'].dt.month
    df = df[df['time'].dt.month.between(6, 10)]
    df = df[df['time'].dt.year.between(start_year, end_year)]
    df[var] = df[var] - 273.15
    return df


In [19]:
df_tmin = preprocess(pd.read_csv(
    "B:\GroundwaterRech\GithubRepo\Restart\Dataset\Final forecasted data 2020-2040\Final forecasted data 2020-2040\SSP126\\tmin.csv"),
    2020, 2035, 'tasmin')

df_tmax = preprocess(pd.read_csv(
    "B:\GroundwaterRech\GithubRepo\Restart\Dataset\Final forecasted data 2020-2040\Final forecasted data 2020-2040\SSP126\\tmax.csv"),
    2020, 2035, 'tasmax')

df_tavg = preprocess(pd.read_csv(
    "B:\GroundwaterRech\GithubRepo\Restart\Dataset\Final forecasted data 2020-2040\Final forecasted data 2020-2040\SSP126\\tavg.csv"),
    2020, 2035, 'tas')

In [20]:
df_tmin

,time,lat,lon,tasmin,month
308568,2020-06-01 12:00:00,29.3,73.55,21.658688,6
308569,2020-06-01 12:00:00,29.3,73.80,21.576681,6
308570,2020-06-01 12:00:00,29.3,74.05,21.494674,6
308571,2020-06-01 12:00:00,29.3,74.30,21.412667,6
308572,2020-06-01 12:00:00,29.3,74.55,21.330660,6
...,...,...,...,...,...
1186999,2035-10-31 12:00:00,32.3,75.30,2.782446,10
1187000,2035-10-31 12:00:00,32.3,75.55,0.398122,10
1187001,2035-10-31 12:00:00,32.3,75.80,-1.986202,10
1187002,2035-10-31 12:00:00,32.3,76.05,-4.370527,10


In [23]:
merged_df = df_tmin.merge(df_tmax, on=['time', 'lat', 'lon']).merge(df_tavg, on=['time', 'lat', 'lon'])
merged_df.drop(['month_x', 'month_y'], axis=1, inplace=True)

In [24]:
merged_df

,time,lat,lon,tasmin,tasmax,tas,month
0,2020-06-01 12:00:00,29.3,73.55,21.658688,36.707586,30.864836,6
1,2020-06-01 12:00:00,29.3,73.80,21.576681,36.665179,30.706705,6
2,2020-06-01 12:00:00,29.3,74.05,21.494674,36.622771,30.548573,6
3,2020-06-01 12:00:00,29.3,74.30,21.412667,36.580364,30.390442,6
4,2020-06-01 12:00:00,29.3,74.55,21.330660,36.537957,30.232310,6
...,...,...,...,...,...,...,...
381883,2035-10-31 12:00:00,32.3,75.30,2.782446,20.150101,11.248109,10
381884,2035-10-31 12:00:00,32.3,75.55,0.398122,17.820745,9.007311,10
381885,2035-10-31 12:00:00,32.3,75.80,-1.986202,15.491389,6.766512,10
381886,2035-10-31 12:00:00,32.3,76.05,-4.370527,13.162033,4.525714,10


In [25]:
factor = {6: 17.0, 7: 16.7, 8: 15.7, 9: 13.9, 10: 11.7}

In [29]:
def calculate_pet(tavg, tmax, tmin, factor):
    return 0.0023 * (tavg + 17.8) * ((tmax - tmin) ** 0.5) * factor

In [30]:
merged_df['PET'] = merged_df.apply(
    lambda row: calculate_pet(row['tas'], row['tasmax'], row['tasmin'], factor[row['month']]), axis=1)

In [32]:
merged_df

,time,lat,lon,tasmin,tasmax,tas,month,PET
0,2020-06-01 12:00:00,29.3,73.55,21.658688,36.707586,30.864836,6,7.381496
1,2020-06-01 12:00:00,29.3,73.80,21.576681,36.665179,30.706705,6,7.367184
2,2020-06-01 12:00:00,29.3,74.05,21.494674,36.622771,30.548573,6,7.352797
3,2020-06-01 12:00:00,29.3,74.30,21.412667,36.580364,30.390442,6,7.338334
4,2020-06-01 12:00:00,29.3,74.55,21.330660,36.537957,30.232310,6,7.323796
...,...,...,...,...,...,...,...,...
381883,2035-10-31 12:00:00,32.3,75.30,2.782446,20.150101,11.248109,10,3.257633
381884,2035-10-31 12:00:00,32.3,75.55,0.398122,17.820745,9.007311,10,3.011090
381885,2035-10-31 12:00:00,32.3,75.80,-1.986202,15.491389,6.766512,10,2.763745
381886,2035-10-31 12:00:00,32.3,76.05,-4.370527,13.162033,4.525714,10,2.515601


In [33]:
merged_df.to_csv(
    "B:\GroundwaterRech\GithubRepo\Restart\Dataset\Final forecasted data 2020-2040\Final forecasted data 2020-2040\SSP126\\pet.csv")